# **Individual Formative assignment January 2025**

## Objectives

* Design and implement an ETL pipeline using Python.
* Visualise data using Matplotlib, Seaborn, and Plotly.
* Adhere to the key milestones and deliverables.
* Maximise future maintainability through documentation, code structure, and organisation.
* Document and present the project process and outcomes.
* Demonstrate and document the development process through a version control system such as GitHub.

Based of the Retail Sales Data Analysis Project Example.  

* Data analysis goals:  
Analyse retail sales data to identify trends, insights, and the impact of promotional markdowns on sales.  
Provide comprehensive, visually appealing sales reports and insights to assist in strategic decision-making.

Context:  

The challenge involves making decisions based on limited historical data, particularly around holidays and promotional events.  
The dataset includes historical sales data for 45 stores in different regions, with details about store types, sizes, and promotional markdowns.  

*  Potential features to include:  
ETL Pipeline:  
    Extract: Load data from the Excel sheets (Stores, Features, Sales).  
    Transform: Clean the data, handle missing values, and create new features such as sales differences between holiday and non-holiday weeks.  
    Load: Store the transformed data in a format suitable for analysis (e.g., a cleaned DataFrame).  
Data Visualisation:  
    Descriptive Statistics: Display basic statistics such as average sales per store and department.  
    Trend Analysis: Plot sales trends over time for different stores and departments.  
    Impact Analysis: Visualise the impact of markdowns on sales during holidays versus non-holiday periods.  
    Comparative Analysis: Compare sales performance across different stores and regions.  

## Inputs

* Data source: https://www.kaggle.com/datasets/manjeetsingh/retaildataset 


## Outputs

* This notebook will be the combined output for the 2 day assignment

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Change working directory

In [226]:
import os
os.chdir('/WORKSPACE/CI_DA_PROJECT_JAN2025_PG')
runtime.txt

FileNotFoundError: [Errno 2] No such file or directory: '/WORKSPACE/CI_DA_PROJECT_JAN2025_PG'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

'/'

Confirm the new current directory

In [225]:
current_dir = os.getcwd()
current_dir

'/'

# ETL - Step 1 data load

Section 1 - setup and import relevant modules

In order to undertake the required tasks the following python modules will be loaded into the Jupyter notebook.


import numpy as np
import pandas as pd
import datetime as dt
import math
import seaborn as sns


In [221]:
# load ETL Modules
import numpy as np
import pandas as pd
import datetime as dt
import math
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)



# Load Original Data

In [222]:
sales_df = pd.read_csv('sales_data-set.csv')
stores_df=  pd.read_csv('stores_data-set.csv')
features_df = pd.read_csv('Features_data_set.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'sales_data-set.csv'

# Starting with sales_df

Review data and enhance it for analysis

In [ ]:
#Headline data for 
Columns = ['Date', 'Dept', 'Store', 'Weekly_Sales', 'IsHoliday']

sales_sum_df = pd.DataFrame({
    'DataType': sales_df[Columns].dtypes,
    'UniqueValues': sales_df[Columns].nunique(),
    'RowsWithData': len(sales_df[Columns])-sales_df[Columns].isnull().sum(),
    'NullValues':sales_df[Columns].isnull().sum(),
    'SkewScore':sales_df[Columns].skew(),
    'Kurtosis':sales_df[Columns].kurtosis()
})
sales_sum_df


In [ ]:
sales_df.head()

## Measures - Sales_data-set.csv

This data set contain the following data elements, there are no Nulls in the data so there is no need for cleansing of Null.  
The date field is not in date format = this needs to be changed.  

sales_data-set.csv  
    Date - Start/end date of week the data is relevant to  
    Store - The store number the data is relevant to - 45 unique stores  
    Dept - The Department within the store that the data is relevant to - 73 unique departments  
    Weekly_sales - Total Weekly Sales for the department and store  
    Is_Holiday = True/False flag that week contained holiday dates  

In [74]:
# Change date to date format and create year/month and week fields and YearMonth YearWeek
# convert Received date to Date Format
sales_df['Date'] = pd.to_datetime(sales_df['Date'], format='%d/%m/%Y')
# Append Year column
sales_df['SalesYear'] = round(sales_df['Date'].dt.year)
# Append Month column 
sales_df['SalesMonth'] = round(sales_df['Date'].dt.month)
# Append Week column
sales_df['SalesWeek'] = round(sales_df['Date'].dt.week)
# Create YearMonth Column
sales_df['SalesYearMonth'] = round(sales_df['SalesYear']*100 +sales_df['SalesMonth'])
# Create YearWeek Column
sales_df['SalesYearWeek'] = round(sales_df['SalesYear']*100 +sales_df['SalesWeek'])


## Full Measures - Sales_data-set.csv

Measures now available  

sales_data-set.csv  
    Date - Start/end date of week the data is relevant to.  
    Store - The store number the data is relevant to - 45 unique stores.  
    Dept - The Department within the store that the data is relevant to - 73 unique departments.  
    Weekly_sales - Total Weekly Sales for the department and store.  
    Is_Holiday = True/False flag that week contained holiday dates.  
    SalesYear = Year the sale was made.  
    SalesMonth - Month the sale was made.  
    SalesWeek - Week the sale was made.  
    SalesYearMonth - Year and Month the sale was made.  
    SalesYearWeek - Year and Week the sale was made.  

In [ ]:
import matplotlib.pyplot as plt

sales_df.query('SalesWeek <= 4')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Create subplots
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(16, 8))

# First graph: Weekly sales over time
sales_df.groupby(['Store', 'Date' ]).Weekly_Sales.sum().unstack(level=0).plot(
    kind='line',
    linestyle='solid',
    legend=False,
    ax=axes[0, 0]
)
axes[0, 0].set_title('Sales per Week')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Weekly Sales')
axes[0, 0].yaxis.set_major_formatter(ticker.EngFormatter())


# Second graph: Total sales per week aggregated by SalesWeek
sales_df.groupby(['SalesYear', 'SalesWeek']).Weekly_Sales.sum().unstack(level=0).plot(
    kind='line',
    linestyle='solid',
    legend=True,
    ax=axes[0, 1]
)
axes[0, 1].set_title('Total Sales per Week: 2010 and 2011')
axes[0, 1].set_xlabel('Sales Week')
axes[0, 1].set_ylabel('Total Weekly Sales')
axes[0, 1].yaxis.set_major_formatter(ticker.EngFormatter())
axes[0, 1].xaxis.set_major_formatter(ticker.EngFormatter())

# Third graph: Mean sales per week aggregated by SalesWeek
sales_df.groupby(['SalesYear', 'SalesWeek']).Weekly_Sales.mean().unstack(level=0).plot(
    kind='line',
    linestyle='solid',
    legend=True,
    ax=axes[1, 0]
)
axes[1, 0].set_title('Mean Sales per Week per Department')
axes[1, 0].set_xlabel('Sales Week')
axes[1, 0].set_ylabel('Mean Department Weekly Sales')
axes[1, 0].yaxis.set_major_formatter(ticker.EngFormatter())
axes[1, 0].xaxis.set_major_formatter(ticker.EngFormatter())

# Fourth graph: Year-on-year comparison of sales
# Group data by year and month, then aggregate sales
monthly_sales = sales_df.groupby(['SalesYear', 'SalesMonth']).Weekly_Sales.sum().unstack(level=0)

# Plot one line per year
monthly_sales.plot(
    kind='line',
    ax=axes[1, 1],
    linestyle='solid',
    marker='o'  # Optional: Adds markers to the line
)

# Update x-axis with month names
axes[1, 1].set_xticks(range(1, 13))  # Months range from 1 to 12
axes[1, 1].set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
axes[1, 1].yaxis.set_major_formatter(ticker.EngFormatter())

# Add title and labels
axes[1, 1].set_title('Year-on-Year Total Sales by Month')
axes[1, 1].set_xlabel('Month')
axes[1, 1].set_ylabel('Total Sales')

# Add legend for years
axes[1, 1].legend(title='Year', loc='upper left')

# Fifth graph: Holiday vs Non Holiday
# Group data by year and Holiday, then aggregate sales
monthly_sales2 = sales_df.groupby(['IsHoliday', 'SalesYear']).Weekly_Sales.mean().unstack(level=0)

monthly_sales2.plot(
    kind='bar',
    ax=axes[2,0],
    color=['skyblue', 'orange'],  # Optional: Custom colors for bars
    edgecolor='black'            # Adds a border to the bars
)

# Add title and labels
axes[2,0].set_title('Average Weekly Sales by Year and Holiday Status')
axes[2,0].set_xlabel('Sales Year')
axes[2,0].set_ylabel('Average Weekly Sales')
axes[2,0].legend(title='Holiday Week', loc='upper left')
axes[2,0].yaxis.set_major_formatter(ticker.EngFormatter())
axes[2,0].xaxis.set_major_formatter(ticker.EngFormatter())

# Sixth graph: Count Departments
# Group data by year and Holiday, then aggregate sales
dept_per_store = sales_df.groupby('Store')['Dept'].nunique()
stores_with_same_no_dept = dept_per_store.value_counts()
stores_with_same_no_dept.sort_index().plot(
    kind='bar',
    x='Dept',
    ax=axes[2,1],
    legend=False,
    edgecolor='black'            # Adds a border to the bars
)

# Add title and labels
axes[2,1].set_title('Departments by Store')
axes[2,1].set_xlabel('Departments in Store')
axes[2,1].set_ylabel('Count of Stores')

# Display the plots
plt.tight_layout()
plt.show()

Lets see box and whisker charts by store

In [ ]:
# Get unique values in RegEntity_Group
unique_groups = sales_df['Store'].unique()

# Number of plots per row
plots_per_row = 6

# Calculate the required number of rows
num_rows = math.ceil(len(unique_groups) / plots_per_row)

# Create subplots
fig, axes = plt.subplots(nrows=num_rows, ncols=plots_per_row, figsize=(18, 3 * num_rows))
axes = axes.flatten()  # Flatten the axes array for easier indexing

# Loop through each unique value in RegEntity_Group and corresponding axes
for idx, group in enumerate(unique_groups):
    # Filter data for the current group
    group_data = sales_df.loc[sales_df['Store'] == group, 'Weekly_Sales'].dropna()
    
    # Create a boxplot for the current group
    axes[idx].boxplot(group_data, vert=True, patch_artist=True, labels=[group])
    
    # Add labels and title
    axes[idx].set_title(f'Store {group}')
    axes[idx].set_ylabel('Weekly_Sales')
    axes[idx].set_xlabel('Store')
    axes[idx].yaxis.set_major_formatter(ticker.EngFormatter())

# Hide any unused axes
for ax in axes[len(unique_groups):]:
    ax.set_visible(False)

# Adjust layout and show plot
plt.yscale('log')
plt.tight_layout()
plt.show()

## Observations on Sales_data-set.csv

The data does not contain 3 complete years, it runs from Jan 2010 to October 2012.  
There is a peak in sales during December in 2010 and 2011.  
There is a corresponding drop in sales in January of 2011 and 2012.  
The 45 Stores appear to have relatively similar sales patterns across the year, although the Size of those sales does vary between store, this is evidently driven by a different factor. 
 This observation is backed up by the variations shown in the Box and Whisker plots by store.  
 Stores received increased sales during holiday weeks on average than during non holiday weeks.  
 Number of deparments varies across Stores from 61 to 79, with most having 77 departments.  No stores have 65-71 departments. 

# Lets look at the other data sets 

Lets see what the Features and Stores data sets add

In [ ]:
#Headline data for 
print(features_df.describe())
features_sum_df = pd.DataFrame({
    'DataType': features_df.dtypes,
    'UniqueValues': features_df.nunique(),
    'RowsWithData': len(features_df)-features_df.isnull().sum(),
    'NullValues': features_df.isnull().sum(),
    'SkewScore': features_df.skew(),
    'Kurtosis': features_df.kurtosis()
})
features_sum_df

## Full Measures - Features_data_set.csv

Measures now available  

Features_data_set.csv
   CPI - Measure of average change over time in prices with 1980 being the base at 100 - 211 means prices are 111% greater than in 1980  
   Date - Start/end date of week the data is relevant to.    
   Fuel_Price - Average Price of fuel per gallon in week  
   IsHoliday - True/False flag showing weeks with a holiday date.  
   MarkDown1 - Value of discounts applied to a product during the period at the identified store  
   MarkDown2 - Value of discounts applied to a product during the period at the identified store  
   MarkDown3 - Value of discounts applied to a product during the period at the identified store  
   MarkDown4 - Value of discounts applied to a product during the period at the identified store  
   MarkDown5 - Value of discounts applied to a product during the period at the identified store  
   Store - Store unique identifier  
   Temperature - Average temperature during period  
   Unemployment - Reported Unemployment rate for period.  

There are Nulls in Markdown, CPI and Unemployment.

CPI and Unemployment are Government measures - the FillNA fill forward method will be used for these  
Markdown is a stated figure of discounts - NaN means no discounts applied - so NaN will be filled with 0.


In [ ]:

# Replace NaN in MarkDown as 0
features_df['MarkDown1'] = features_df['MarkDown1'].fillna(0)
features_df['MarkDown2'] = features_df['MarkDown2'].fillna(0)
features_df['MarkDown3'] = features_df['MarkDown3'].fillna(0)
features_df['MarkDown4'] = features_df['MarkDown4'].fillna(0)
features_df['MarkDown5'] = features_df['MarkDown5'].fillna(0)
features_df['MarkDownTotal']= features_df['MarkDown1']+features_df['MarkDown2']+features_df['MarkDown3']+features_df['MarkDown4']+features_df['MarkDown5']
features_df['MarkDownData']= features_df['MarkDownTotal']>0
# Use fillna forward on Unemployment and CPI
features_df['CPI']=features_df['CPI'].fillna(method='ffill')
features_df['Unemployment']=features_df['Unemployment'].fillna(method='ffill')

# re run summary
features_pc_sum_df = pd.DataFrame({
    'DataType': features_df.dtypes,
    'UniqueValues': features_df.nunique(),
    'RowsWithData': len(features_df)-features_df.isnull().sum(),
    'NullValues': features_df.isnull().sum(),
    'SkewScore': features_df.skew(),
    'Kurtosis': features_df.kurtosis()
})
features_pc_sum_df

# 

In [ ]:
#Headline data for 
print(stores_df.describe())
stores_sum_df = pd.DataFrame({
    'DataType': stores_df.dtypes,
    'UniqueValues': stores_df.nunique(),
    'RowsWithData': len(stores_df)-stores_df.isnull().sum(),
    'NullValues': stores_df.isnull().sum(),
    'SkewScore': stores_df.skew(),
    'Kurtosis': stores_df.kurtosis()
})
stores_sum_df

In [ ]:
stores_df.head()

## Full Measures - Stores_data_set.csv

Measures available  

Stores_data_set.csv  
    Store - Store unique Id   
    Type - Store classification - choice of A, B or C  
    Size - Store floor space in Sqft.  

There are no Nulls that need cleaning.


In [ ]:
# Graphs from Stores - Code Corrected using ChatGpt
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16, 6))

stores_df['Size'].plot(kind='hist', ax=axes[0], bins=20, edgecolor='black')
axes[0].set_title('Distribution of Store Sizes')
axes[0].set_xlabel('Size')
axes[0].set_ylabel('Frequency')

stores_df['Type'].value_counts().plot(kind='bar', ax=axes[1], color='lightgreen', edgecolor='black')
axes[1].set_title('Store Count by Type')
axes[1].set_xlabel('Type')
axes[1].set_ylabel('Count')

stores_df.plot(kind='scatter', x='Type', y='Size', ax=axes[2], color='orange')
axes[2].set_title('Size vs Type')
axes[2].set_xlabel('Type')
axes[2].set_ylabel('Size')
axes[2].yaxis.set_major_formatter(ticker.EngFormatter())

plt.tight_layout()
plt.show()

## Enhance stores data with Count of Department and see if there is a relation between type and Number of Departments.

Using the summary of department for store created earlier to produce the graph of count of stores by number of departments, the Stores_data-csv will be enhanced.

In [ ]:
stores_df

In [ ]:
# merege the additional data on departments into data set so that it will be included further on
stores_df=pd.merge(stores_df, dept_per_store, how='left', on=['Store'])

# Graphs from Stores enhanced with number of departments
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16, 6))

stores_df.plot(kind='scatter', x='Size', y='Dept')
axes[0].set_title('Distribution of Store Sizes by number of departments')
axes[0].set_xlabel('Size')
axes[0].set_ylabel('Departments')

stores_df.plot(kind='scatter', x='Type', y='Dept')
axes[1].set_title('Departments by Type')
axes[1].set_xlabel('Type')
axes[1].set_ylabel('Departments')

stores_df.plot(kind='scatter', x='Store', y='Dept', ax=axes[2], color='orange')
axes[2].set_title('Size vs Type')
axes[2].set_xlabel('Type')
axes[2].set_ylabel('Size')
axes[2].yaxis.set_major_formatter(ticker.EngFormatter())

plt.tight_layout()
plt.show()

## Observations Regarding the Stores Data

There are significantly less Type C stores than those of A or B.  
Store Type appears to be classified by Size with a few outliers in the A and B category.  
Type potentially could be reclassfied and this presents a future option.  

# Append the Store Data into the Sales Data to enable groupby type and size also to look as sales by size

In [ ]:
enhancedsales_df=pd.merge(sales_df, stores_df, how='left', on=['Store'])
enhancedsales_df.head()

In [ ]:
# Add Sales/size of store to produce a normalised sales per sqft figure
enhancedsales_df['SalesPerSqFt']=enhancedsales_df['Weekly_Sales']/enhancedsales_df['Size']
enhancedsales_df.head()

In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(16, 8))

# First graph: Weekly sales over time
enhancedsales_df.groupby(['Store', 'Date' ]).SalesPerSqFt.sum().unstack(level=0).plot(
    kind='line',
    linestyle='solid',
    legend=False,
    ax=axes[0, 0]
)
axes[0, 0].set_title('SalesPerSqFt per Week')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('SalesPerSqFt')

# Second graph: Total sales per week aggregated by SalesWeek
enhancedsales_df.groupby(['SalesYear', 'SalesWeek']).SalesPerSqFt.sum().unstack(level=0).plot(
    kind='line',
    linestyle='solid',
    legend=True,
    ax=axes[0, 1]
)
axes[0, 1].set_title('SalesPerSqFt per Week: 2010 and 2011')
axes[0, 1].set_xlabel('Sales Week')
axes[0, 1].set_ylabel('Total SalesPerSqFt')

# Third graph: Mean sales per week aggregated by SalesWeek
enhancedsales_df.groupby(['SalesYear', 'SalesWeek']).SalesPerSqFt.mean().unstack(level=0).plot(
    kind='line',
    linestyle='solid',
    legend=True,
    ax=axes[1, 0]
)
axes[1, 0].set_title('Mean SalesPerSqFt per Department')
axes[1, 0].set_xlabel('Sales Week')
axes[1, 0].set_ylabel('Mean Department SalesPerSqFt Sales')

# Fourth graph: Year-on-year comparison of sales
# Group data by year and month, then aggregate sales
monthly_sales = enhancedsales_df.groupby(['SalesYear', 'SalesMonth']).SalesPerSqFt.sum().unstack(level=0)

# Plot one line per year
monthly_sales.plot(
    kind='line',
    ax=axes[1, 1],
    linestyle='solid',
    marker='o'  # Optional: Adds markers to the line
)

# Update x-axis with month names
axes[1, 1].set_xticks(range(1, 13))  # Months range from 1 to 12
axes[1, 1].set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

# Add title and labels
axes[1, 1].set_title('Year-on-Year SalesPerSqFt by Month')
axes[1, 1].set_xlabel('Month')
axes[1, 1].set_ylabel('SalesPerSqFt')

# Add legend for years
axes[1, 1].legend(title='Year', loc='upper left')

# Fifth graph: Holiday vs Non Holiday
# Group data by year and Holiday, then aggregate sales
monthly_sales = enhancedsales_df.groupby(['IsHoliday', 'SalesYear']).SalesPerSqFt.mean().unstack(level=0)

monthly_sales.plot(
    kind='bar',
    ax=axes[2,0],
    color=['skyblue', 'orange'],  # Optional: Custom colors for bars
    edgecolor='black'            # Adds a border to the bars
)

# Add title and labels
axes[2,0].set_title('Average SalesPerSqFt by Year and Holiday Status')
axes[2,0].set_xlabel('Sales Year')
axes[2,0].set_ylabel('Average SalesPerSqFt')
axes[2,0].legend(title='Holiday Week', loc='upper left')

# Display the plots
plt.tight_layout()
plt.show()

## Observations on Sales Per Size

# Append the Features Data into the Enhanced Sales Data to enable groupby type and size also to look as sales by size

In [ ]:
# Change date on features_df to a dateformat.
features_df['Date'] = pd.to_datetime(features_df['Date'], format='%d/%m/%Y')
# merge enhanced and feature based on date and store
enhancedsales_df=pd.merge(enhancedsales_df, features_df, how='left', on=['Store','Date'])
enhancedsales_df.head()

In [ ]:
#create a summary of sales per store per week
# function_dictionary = {'OrderID':'count','Quantity':'mean'}
salesaggregate = {'Weekly_Sales':'sum', 'SalesPerSqFt':'sum', 'Size': 'max', 'Temperature': 'max', 'Fuel_Price': 'max', 'Dept': 'count', 'Unemployment': 'max', 'IsHoliday_x': 'max', 'SalesYear': 'max', 'SalesWeek': 'max', 'SalesMonth': 'max', 'Type': 'first', 'CPI': 'max', 'MarkDownTotal': 'max', 'MarkDownData': 'max'}
storesalessummary = enhancedsales_df.groupby(['Store','Date'], as_index=False).agg(salesaggregate).reset_index()


storesalessummary['NonMarkedDownSales'] = storesalessummary['Weekly_Sales'] - storesalessummary['MarkDownTotal']
storesalessummary['PctSalesMarkedDown'] = storesalessummary['MarkDownTotal']/storesalessummary['Weekly_Sales']
storesalessummary['MD_PerSqFt'] = storesalessummary['MarkDownTotal']/storesalessummary['Size']
storesalessummary['NMDS_PerSqFt'] = storesalessummary['NonMarkedDownSales']/ storesalessummary['Size']

storesalessummary.head(5)

In [ ]:
# ReRun Box and Whisker per store

# Get unique values in RegEntity_Group
unique_groups = storesalessummary['Store'].unique()

# Number of plots per row
plots_per_row = 6

# Calculate the required number of rows
num_rows = math.ceil(len(unique_groups) / plots_per_row)

# Create subplots
fig, axes = plt.subplots(nrows=num_rows, ncols=plots_per_row, figsize=(18, 3 * num_rows))
axes = axes.flatten()  # Flatten the axes array for easier indexing

# Loop through each unique value in RegEntity_Group and corresponding axes
for idx, group in enumerate(unique_groups):
    # Filter data for the current group
    group_data = storesalessummary.loc[storesalessummary['Store'] == group, 'SalesPerSqFt'].dropna()
    
    # Create a boxplot for the current group
    axes[idx].boxplot(group_data, vert=True, patch_artist=True, labels=[group])
    
    # Add labels and title
    axes[idx].set_title(f'Strore {group}')
    axes[idx].set_ylabel('SalesPerSqFt')
    axes[idx].set_xlabel('Store')

# Hide any unused axes
for ax in axes[len(unique_groups):]:
    ax.set_visible(False)

# Adjust layout and show plot
plt.yscale('log')
plt.tight_layout()
plt.show()

## Observations on Sales By Store Type

Unsuprisingly the large stores on average have higher weekly sales both during Holiday and Non Holiday weeks.  
However Type C the smaller stores and Type B the Mid range stores - during non holiday weeks have higher Average Sales per Sq Ft than the larger Type A stores.  
During Holiday periods Type B Stores have a similar Average sales per sq ft at Type A but their distributed sales per sq ft are generally higher.  
Type C out performs both Type B and Type A in Sales Per Sq Ft during Holiday weeks.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16, 6))

Weekly_storesalessummary = pd.pivot_table(storesalessummary, index ='SalesWeek', columns = 'SalesYear', values = 'Weekly_Sales', aggfunc='mean')
Weekly_storesalessummary.plot(ax=axes[0])
axes[0].set_title('Avg Store Sales')
axes[0].yaxis.set_major_formatter(ticker.EngFormatter())
Weekly_storesalessummary2 = storesalessummary.query('MarkDownData == True').pivot_table(index ='SalesMonth', columns = 'SalesYear', values = 'NonMarkedDownSales', aggfunc='mean')
Weekly_storesalessummary2.plot(ax=axes[1])
axes[1].set_title('Avg Store Non Discounted Sales')
axes[1].yaxis.set_major_formatter(ticker.EngFormatter())
Weekly_storesalessummary3 = storesalessummary.query('MarkDownData == True').pivot_table(index ='SalesMonth', columns = 'SalesYear', values = 'MarkDownTotal', aggfunc='mean')
Weekly_storesalessummary3.plot(ax=axes[2])
axes[2].set_title('Avg Store Discounted Sales')
axes[2].yaxis.set_major_formatter(ticker.EngFormatter())

In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(16, 10))

# First plot
sns.violinplot(
    x='Type', y='Weekly_Sales', data=storesalessummary[['Weekly_Sales', 'Type']],
    ax=axes[0, 0], scale='width'  # Assign the specific subplot axis
)
axes[0, 0].set_title('Comparing Store Sales by Type')
axes[0, 0].set_ylim(-250000, 4250000)
axes[0, 0].yaxis.set_major_formatter(ticker.EngFormatter())
 

# Second plot
sns.violinplot(
    x='Type', y='SalesPerSqFt', data=storesalessummary[['SalesPerSqFt', 'Type']],
    ax=axes[1, 0], scale='width'  # Assign the specific subplot axis
)
axes[1, 0].set_title('Comparing Store SalesPerSqFt by Type')
axes[1, 0].set_ylim(0, 33) 
axes[1, 0].yaxis.set_major_formatter(ticker.EngFormatter())

# third plot = Weekly Sales for Holiday Weeks
sns.violinplot(
    x='Type', y='Weekly_Sales', data=storesalessummary.query('IsHoliday_x == True')[['Weekly_Sales', 'Type']],
    ax=axes[0, 1], scale='width'  # Assign the specific subplot axis
)
axes[0, 1].set_title('Comparing Store Holiday Sales by Type')
axes[0, 1].set_ylim(-250000, 4250000) 
axes[0, 1].yaxis.set_major_formatter(ticker.EngFormatter())

# Forth plot = Weekly SalesPerSqFt for Holiday Weeks
sns.violinplot(
    x='Type', y='SalesPerSqFt', data=storesalessummary.query('IsHoliday_x == True')[['SalesPerSqFt', 'Type']],
    ax=axes[1, 1], scale='width'  # Assign the specific subplot axis
)
axes[1, 1].set_title('Comparing Store Holiday SalesPerSqFt by Type')
axes[1, 1].set_ylim(0, 33)
axes[1, 1].yaxis.set_major_formatter(ticker.EngFormatter())

# fifth plot = Weekly Sales for Non Holiday Weeks
sns.violinplot(
    x='Type', y='Weekly_Sales', data=storesalessummary.query('IsHoliday_x == False')[['Weekly_Sales', 'Type']],
    ax=axes[0, 2], scale='width'  # Assign the specific subplot axis
)
axes[0, 2].set_title('Comparing Store Non Holiday Sales by Type')
axes[0, 2].set_ylim(-250000, 4250000) 
axes[0, 2].yaxis.set_major_formatter(ticker.EngFormatter())

# Six plot = Weekly SalesPerSqFt for Non Holiday Weeks
sns.violinplot(
    x='Type', y='SalesPerSqFt', data=storesalessummary.query('IsHoliday_x == False')[['SalesPerSqFt', 'Type']],
    ax=axes[1, 2], scale='width'  # Assign the specific subplot axis
)
axes[1, 2].set_title('Comparing Store Non Holiday SalesPerSqFt by Type')
axes[1, 2].set_ylim(0, 33)
axes[1, 2].yaxis.set_major_formatter(ticker.EngFormatter())

# Adjust layout
plt.tight_layout()
plt.show()

---

# Store and Department level performance information and Graphs.  

Using the sales_df at weekly sales per department level produce a set of visualisations to enable store managers to review store specific performance by department over time

In [ ]:
# Calculate Delta fields for Weekly Sales both Year on Year and Week on Week, aslo create moving 13 week average.

# Add rolling 4 and 13 week total sales
sales_df['Dept_Roll_4wk_sales'] = sales_df.sort_values(['Store','Dept','Date'])['Weekly_Sales'].rolling(4).sum()
sales_df['Dept_Roll_Qtr_sales'] = sales_df.sort_values(['Store','Dept','Date'])['Weekly_Sales'].rolling(13).sum()

# week on week PCT change per store department
sales_df['Dept_weekly_Growth_Rate'] = sales_df.sort_values(['Store','Dept','Date'])['Weekly_Sales'].pct_change(periods=1) * 100
# Year on Year PCT Change per store department
sales_df['Dept_yearly_Growth_Rate'] = sales_df.sort_values(['Store','Dept','Date'])['Weekly_Sales'].pct_change(periods=53) * 100
# week on week PCT change per store department for 4 week rolling and 13 week rolling
sales_df['Dept_4wk_roll_Growth_Rate'] = sales_df.sort_values(['Store','Dept','Date'])['Dept_Roll_4wk_sales'].pct_change(periods=1) * 100
sales_df['Dept_13wk_roll_Growth_Rate'] = sales_df.sort_values(['Store','Dept','Date'])['Dept_Roll_Qtr_sales'].pct_change(periods=1) * 100

sales_df.head()


In [ ]:
# Get unique values in RegEntity_Group
unique_groups = storesalessummary['Type'].unique()

# Number of plots per row
plots_per_row = 3

# Calculate the required number of rows
num_rows = math.ceil(len(unique_groups) / plots_per_row)

# Create subplots
fig, axes = plt.subplots(nrows=num_rows, ncols=plots_per_row, figsize=(18, 6 * num_rows))
axes = axes.flatten()  # Flatten the axes array for easier indexing

# Loop through each unique value in RegEntity_Group and corresponding axes
for idx, group in enumerate(unique_groups):
    # Filter data for the current group
    group_data = storesalessummary.loc[storesalessummary['Type'] == group, 'Weekly_Sales'].dropna()
    
    # Create a boxplot for the current group
    axes[idx].boxplot(group_data, vert=True, patch_artist=True, labels=[group])
    
    # Add labels and title
    axes[idx].set_title(f'Boxplot of Value for {group}')
    axes[idx].set_ylabel('Weekly_Sales')
    axes[idx].set_xlabel('Store Type')
    axes[idx].set_ylim(-250000 , 4250000)

# Hide any unused axes
for ax in axes[len(unique_groups):]:
    ax.set_visible(False)

# Adjust layout and show plot
plt.tight_layout()
plt.show()

# Get unique values in RegEntity_Group
unique_groups = storesalessummary['Type'].unique()

# Number of plots per row
plots_per_row = 3

# Calculate the required number of rows
num_rows = math.ceil(len(unique_groups) / plots_per_row)

# Create subplots
fig, axes = plt.subplots(nrows=num_rows, ncols=plots_per_row, figsize=(18, 6 * num_rows))
axes = axes.flatten()  # Flatten the axes array for easier indexing

# Loop through each unique value in RegEntity_Group and corresponding axes
for idx, group in enumerate(unique_groups):
    # Filter data for the current group
    group_data = storesalessummary.loc[storesalessummary['Type'] == group, 'MarkDownTotal'].dropna()
    
    # Create a boxplot for the current group
    axes[idx].boxplot(group_data, vert=True, patch_artist=True, labels=[group])
    
    # Add labels and title
    axes[idx].set_title(f'Boxplot of Value for {group}')
    axes[idx].set_ylabel('MarkDownTotal')
    axes[idx].set_xlabel('Store Type')

# Hide any unused axes
for ax in axes[len(unique_groups):]:
    ax.set_visible(False)

# Adjust layout and show plot
plt.tight_layout()
plt.show()

In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(18, 18))

storesalessummary.query('MarkDownTotal >0').plot(kind='scatter', x='Weekly_Sales', y='MarkDownTotal', c='Store', colormap='viridis', ax=axes[0, 0])
axes[0, 0].set_title('Weekly Sales vs Discount value')
axes[0, 0].set_ylabel('Total Discount Applied')
axes[0, 0].set_xlabel('Total Weekly Sales')
axes[0, 0].xaxis.set_major_formatter(ticker.EngFormatter())
axes[0, 0].yaxis.set_major_formatter(ticker.EngFormatter())

storesalessummary.query('MarkDownTotal >0').plot(kind='scatter', x='Weekly_Sales', y='PctSalesMarkedDown', c='Store', colormap='viridis', ax=axes[0, 1])
axes[0, 1].set_title('Weekly Sales vs % of Sales discounted')
axes[0, 1].set_ylabel('Percentage Discounted')
axes[0, 1].set_xlabel('Total Weekly Sales')
axes[0, 1].xaxis.set_major_formatter(ticker.EngFormatter())
axes[0, 1].yaxis.set_major_formatter(ticker.PercentFormatter(xmax =1))

storesalessummary.query('MarkDownTotal >0').plot(kind='scatter', x='MarkDownTotal', y='PctSalesMarkedDown', c='Store', colormap='viridis', ax=axes[0, 2])
axes[0, 2].set_title('Discount Sales vs % of Sales Discounted')
axes[0, 2].set_ylabel('Percentage Discounted')
axes[0, 2].set_xlabel('Total Discounted Sales')
axes[0, 2].xaxis.set_major_formatter(ticker.EngFormatter())
axes[0, 2].yaxis.set_major_formatter(ticker.PercentFormatter(xmax =1))

storesalessummary.query('IsHoliday_x == True').plot(kind='scatter', x='Weekly_Sales', y='MarkDownTotal', c='PctSalesMarkedDown', colormap='viridis', ax=axes[1, 0])
axes[1, 0].set_title('Weekly Sales vs Discount value for Holiday Weeks')
axes[1, 0].set_ylabel('Total Discount Applied')
axes[1, 0].set_xlabel('Total Weekly Sales')
axes[1, 0].xaxis.set_major_formatter(ticker.EngFormatter())
axes[1, 0].yaxis.set_major_formatter(ticker.EngFormatter())

storesalessummary.query('IsHoliday_x == True').plot(kind='scatter', x='Weekly_Sales', y='PctSalesMarkedDown', c='MarkDownTotal', colormap='viridis', ax=axes[1, 1])
axes[1, 1].set_title('Weekly Sales vs % of Sales Discounted holiday weeks')
axes[1, 1].set_ylabel('Percentage Discounted')
axes[1, 1].set_xlabel('Total Weekly Sales')
axes[1, 1].xaxis.set_major_formatter(ticker.EngFormatter())
axes[1, 1].yaxis.set_major_formatter(ticker.PercentFormatter(xmax =1))


storesalessummary.query('IsHoliday_x == True').plot(kind='scatter', x='MarkDownTotal', y='PctSalesMarkedDown', c='Weekly_Sales', colormap='viridis', ax=axes[1, 2])
axes[1, 2].set_title('Discount Sales vs % of Sales Discounted holiday weeks')
axes[1, 2].set_ylabel('Percentage Discounted')
axes[1, 2].set_xlabel('Total Discounted Sales')
axes[1, 2].xaxis.set_major_formatter(ticker.EngFormatter())
axes[1, 2].yaxis.set_major_formatter(ticker.PercentFormatter(xmax =1))

storesalessummary.query('IsHoliday_x == False').plot(kind='scatter', x='Weekly_Sales', y='MarkDownTotal', c='PctSalesMarkedDown', colormap='viridis', ax=axes[2, 0])
axes[2, 0].set_title('Weekly Sales vs Discount value Non Holiday weeks')
axes[2, 0].set_ylabel('Total Discount Applied')
axes[2, 0].set_xlabel('Total Weekly Sales')
axes[2, 0].xaxis.set_major_formatter(ticker.EngFormatter())
axes[2, 0].yaxis.set_major_formatter(ticker.EngFormatter())

storesalessummary.query('IsHoliday_x == False').plot(kind='scatter', x='Weekly_Sales', y='PctSalesMarkedDown', c='MarkDownTotal', colormap='viridis', ax=axes[2, 1])
axes[2, 1].set_title('Weekly Sales vs % of Sales Marked down Non holiday week')
axes[2, 1].set_ylabel('Percentage Discounted')
axes[2, 1].set_xlabel('Total Weekly Sales')
axes[2, 1].xaxis.set_major_formatter(ticker.EngFormatter())
axes[2, 1].yaxis.set_major_formatter(ticker.PercentFormatter(xmax =1))

storesalessummary.query('IsHoliday_x == False').plot(kind='scatter', x='MarkDownTotal', y='PctSalesMarkedDown', c='Weekly_Sales', colormap='viridis', ax=axes[2, 2])
axes[2, 2].set_title('Discount Sales vs % of Sales Discounted Non holiday weeks')
axes[2, 2].set_ylabel('Percentage Discounted')
axes[2, 2].set_xlabel('Total Discounted Sales')
axes[2, 2].xaxis.set_major_formatter(ticker.EngFormatter())
axes[2, 2].yaxis.set_major_formatter(ticker.PercentFormatter(xmax =1))

plt.tight_layout()
plt.show()

## Improve graphs by adding trend Line

## To do this need to use Seaborn instead of Pandas.plot/

## Observations on relationships Between Weekly Sales, Total Discounted Sales and Percentage Discount Represented

There appears to be a direct correlation between the size of the Percentage Discount applied and the volume of Discount Sales.  
There is also a positive correlation between Total Sales and Percentage Discount applied.  This suggests that the greater the discount on sales the greater overall sales are.  

Both of these observations are true for both Holiday and Non Holiday weeks.



---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Conclusion and Next Steps

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.